In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from googlesearch import search
import pandas as pd
import numpy as np
from datetime import datetime
import re
import time

test_ua = 'Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36'

options = Options()

# options.add_argument("--headless")  # Remove this if you want to see the browser (Headless makes the chromedriver not have a GUI)
options.add_argument("--window-size=1920,1080")

options.add_argument(f'--user-agent={test_ua}')

options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options = options)  # or your browser of choice




In [13]:
path = "../Documents/Ticket Sales.xlsx"
sales = pd.read_excel(path, sheet_name ="Sheet1")


In [14]:
driver.get("https://www.stubhub.ca")

driver.find_element(By.XPATH, "//*[text() ='Sign In']").click()
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='email']")))
driver.find_element(By.CSS_SELECTOR, "input[type='email']").send_keys("eric9090909090@hotmail.com")
driver.find_element(By.CSS_SELECTOR, "input[type=password]").send_keys("BlckPnk39!@!"+ Keys.ENTER)
time.sleep(5)
try:
    driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()
except:
    pass
time.sleep(5)

In [15]:
def stub_url(s):
    if s in [None, np.nan]:
        return "lol"
    else:
        s = s.replace(" ", "+")
        return "https://www.stubhub.ca/secure/search?q="+s+ " Toronto"+"&sellSearch=false&sortBy="



In [16]:
path = "../Documents/Ticket Sales.xlsx"
sales = pd.read_excel(path, sheet_name ="Sheet1")

In [17]:
names = sales["Artist"]

remain = []
prices = []
my_prices = []
dates  = []

today = datetime.today()

for i,r in sales.iterrows():

    if  today < r["Date"] and r["Artist"] not in remain:
        artist_search = stub_url(r["Artist"])
        driver.get(artist_search)
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div[4]/div[2]/div/div[1]/div/div[2]/ul/li[2]/a")))
            event_link = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[4]/div[2]/div/div[1]/div/div[2]/ul/li[2]/a").get_attribute("href")+"&betterValueTickets=false"+ "&estimatedFees=false"
            driver.get(event_link)
        except:
            print(r["Artist"] + ' no event link')
            continue
        driver.find_element(By.XPATH, '//*[@id="modal-root"]/div/div/div/div[2]/div[3]/button').click()
        #filter
        driver.find_element(By.CSS_SELECTOR, "div.sc-xrltsx-2").click()
        time.sleep(1)
        if (driver.find_element(By.CSS_SELECTOR, "input[type=checkbox]").get_attribute("value") == "true"):
            driver.find_element(By.CSS_SELECTOR, "input[type=checkbox]").click()
        time.sleep(4)
        me_price = 0
        listings = driver.find_elements(By.CLASS_NAME, "sc-57jg3s-0")
        for l in listings:
            try:
                l.find_element(By.CLASS_NAME, "sc-kj9927-14")
                me_price = l.find_element(By.CLASS_NAME,  "sc-1bp3ico-0").get_attribute("data-price")
                break
            except:
                pass
        if me_price == 0:
            print(r["Artist"] + ' no me price')
        try:
            price = driver.find_element(By.CLASS_NAME,  "sc-1bp3ico-0").get_attribute("data-price")
            remain.append(r["Artist"])
            prices.append(price)
            my_prices.append(me_price)
            dates.append(r["Date"])
            print(r["Artist"], price, me_price)  
        except:
            print('no price')
            pass

       


JpegMafia C$61 C$67
Porter Robinson no me price
Porter Robinson C$63 0
Kygo no me price
Kygo C$109 0
Conan Grey no me price
Conan Grey C$13 0
Usher no me price
Usher C$37 0
Vampire Weekeend no me price
Vampire Weekeend C$14 0
Korn no me price
Korn C$106 0
Boris Bredja no me price
Boris Bredja C$112 0
Kaytrana no me price
Kaytrana C$27 0
Twenty One Pilots no me price
Twenty One Pilots C$112 0
Meghan Trainor no me price
Meghan Trainor C$68 0
Shaq C$76 C$76
Billie Eillish no me price
Billie Eillish C$214 0
Fred Agin no me price
Fred Agin C$67 0
Keshi no me price
Keshi C$62 0
Justin Timberlake no me price
Justin Timberlake C$66 0
Suicide Boys no me price
Suicide Boys C$214 0
Myles Smith no me price
Myles Smith C$88 0
Timmy Trumpet C$73 C$73
Pitbull no me price
Pitbull C$196 0
Polo G no me price
Polo G C$46 0
Maddix no me price
Maddix C$944 0
Vini Vici no me price
Vini Vici C$57 0
Tinashe C$58 C$68
Sullivan King C$75 C$84
Virtual Riot no me price
Virtual Riot C$58 0
Kiss of Life no me price

In [18]:
stubhub = pd.concat([pd.Series(remain), pd.Series(dates), pd.Series(prices), pd.Series(my_prices),],axis=1)
stubhub.columns = ["Artist","Dates", "Stubhub", "Me"]
stubhub["Stubhub"] = stubhub["Stubhub"].str.extract('(\\d+)').astype(float)
stubhub["Me"] = stubhub["Me"].str.extract('(\\d+)').astype(float)
stubhub = stubhub.sort_values(by="Me", ascending=False)
with pd.ExcelWriter(path,  mode='a',  engine="openpyxl", if_sheet_exists="replace") as writer:
    stubhub.to_excel(writer, sheet_name = "stubhub", header=True)

In [19]:
stubhub[stubhub['Stubhub'] < stubhub['Me']]


,Artist,Dates,Stubhub,Me
36,Allan Walker,2025-02-07,86.0,87.0
24,Sullivan King,2024-11-15,75.0,84.0
23,Tinashe,2024-11-11,58.0,68.0
0,JpegMafia,2024-09-19,61.0,67.0
35,Lavern,2024-11-01,58.0,60.0
30,Atliens,2024-11-16,51.0,54.0


In [20]:
artist_search = stub_url("Jpegmafia")
driver.get(artist_search)
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div[4]/div[2]/div/div[1]/div/div[2]/ul/li[2]/a")))
    event_link = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[4]/div[2]/div/div[1]/div/div[2]/ul/li[2]/a").get_attribute("href")+"&betterValueTickets=false"+ "&estimatedFees=false"
    driver.get(event_link)
except:
    print(r["Artist"] + ' no event link')
    
driver.find_element(By.XPATH, '//*[@id="modal-root"]/div/div/div/div[2]/div[3]/button').click()
#filter
driver.find_element(By.CSS_SELECTOR, "div.sc-xrltsx-2").click()
time.sleep(1)
if (driver.find_element(By.CSS_SELECTOR, "input[type=checkbox]").get_attribute("value") == "true"):
    driver.find_element(By.CSS_SELECTOR, "input[type=checkbox]").click()
time.sleep(4)
me_price = 0
listings = driver.find_elements(By.CLASS_NAME, "sc-57jg3s-0")
for l in listings:
    try:
        l.find_element(By.CLASS_NAME, "sc-kj9927-14")
        me_price = l.find_element(By.CLASS_NAME,  "sc-hlalgf-0.sc-hlalgf-1").text
        break
    except:
        pass
if me_price == 0:
    print(' no me price')
try:
    price = driver.find_element(By.CLASS_NAME,  "sc-hlalgf-0.sc-hlalgf-1").text
    # remain.append(r["Artist"])
    # prices.append(price)
    # my_prices.append(me_price)
    # dates.append(r["Date"])
    print(price, me_price)  
except:
    print('no price')
    pass




C$61 C$67


In [24]:
me_price = 0
listings = driver.find_elements(By.CLASS_NAME, "sc-57jg3s-0")
for l in listings:
    try:
        l.find_element(By.CLASS_NAME, "sc-kj9927-14")
        me_price = l.find_element(By.CLASS_NAME, "sc-1bp3ico-0").get_attribute("data-price")
        break
    except:
        pass
if me_price == 0:
    print(' no me price')
try:
    price = driver.find_element(By.CLASS_NAME, "sc-1bp3ico-0").get_attribute("data-price")
    print(price, me_price)  
except:
    print('no price')
    pass

C$61 C$67


In [22]:
print(len(listings))

46
